<a href="https://colab.research.google.com/github/KoMurase/movie_research_2/blob/master/Preprocessing_Title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [170]:
!cp -r '/content/drive/MyDrive/Movies/data/mojo_japan/' '/content/'

!cp -r '/content/drive/MyDrive/Movies/data/metadata_filmarks/' '/content/'

In [171]:
!pip install texthero
!pip install textdistance

In [172]:
!pip install japanize_matplotlib
import pandas as pd
import codecs 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import japanize_matplotlib

import warnings

import os
from tqdm import tqdm

In [173]:
def read_data(dir, encode_type):
  output=pd.DataFrame()
  print(dir)
  files=os.listdir(dir)
  print(files)

  for file_ in files:
    if '2017' or '2018' or '2019' in file_: #収集できているファイル


      path = dir+file_
      print(path)
      with codecs.open(path, "r", encode_type, "ignore") as file:
          d = pd.read_csv(file, delimiter=",")
      
      output = pd.concat([output, d], axis=0)

  return output

In [174]:
metadata_dir='/content/metadata_filmarks/'
mojo_dir='/content/mojo_japan/'

data = read_data(metadata_dir, encode_type='utf-8')
mdata = read_data(mojo_dir, encode_type='shift-jis')
mdata = mdata.rename(columns={'Filmarks URL':'URL'})
df = data.merge(mdata, how='left',on='URL').reset_index(drop=True)
drop_cols=[
           'url','Release Date','タイトル(日本名)','タイトル(英名)', 
           'header情報',  'あらすじ'
]
df = df.drop(drop_cols, axis=1) 

/content/metadata_filmarks/
['data_2019_2020_12_16_204521.csv', 'data_2017_2020_11_25_015410.csv', 'data_2018_2020_12_14_020607.csv']
/content/metadata_filmarks/data_2019_2020_12_16_204521.csv
/content/metadata_filmarks/data_2017_2020_11_25_015410.csv
/content/metadata_filmarks/data_2018_2020_12_14_020607.csv
/content/mojo_japan/
['data_2017.csv', 'data_2019_2020_12_17_102319.csv', 'data_2002_2020_12_17_101219.csv', 'data_2007_2020_12_17_101219.csv', 'data_2016_2020_12_17_101547.csv', 'data_2012_2020_12_17_101219.csv', 'data_2010_2020_12_17_102203.csv', 'data_2015_2020_12_17_101219.csv', 'data_2003_2020_12_17_101219.csv', 'data_2005_2020_12_17_101219.csv', 'data_2009_2020_12_17_101219.csv', 'data_2013_2020_12_17_102223.csv', 'data_2006_2020_12_17_101219.csv', 'data_2011_2020_12_17_101219.csv', 'data_2008_2020_12_17_101219.csv', 'data_2017_2020_12_17_102319.csv', 'data_2020_2020_12_17_102738.csv', 'data_2004_2020_12_17_101219.csv', 'data_2019.csv', 'data_2014_2020_12_17_101219.csv', 'da

In [175]:
df=df[df['Title'].duplicated()==False]
df.shape

(518, 23)

In [176]:
from nltk.util import ngrams
import texthero as hero
from texthero import preprocessing

def line_ngram(line, n=2):
  words = [w for w in line.split(' ') if len(w) != 0]
  return list(ngrams(words, n))

def clearn_by_hero(x):
    custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   , preprocessing.remove_whitespace
                   , preprocessing.remove_stopwords
                  ]
    return hero.clean(x, custom_pipeline)
    
def create_n_gram(x, flg=False, n=3): 
    if flg==True:
      x = clearn_by_hero(x)
    x = pd.Series(x).map(lambda r: line_ngram(r, n=n))
    return x

In [177]:
from textdistance import jaro

In [178]:
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   , preprocessing.remove_whitespace
                   , preprocessing.remove_stopwords
                  ]

In [179]:
df['Title'] = hero.clean(df['Title'], custom_pipeline)
table = pd.DataFrame(df['Title'])
title = df['Title'].values
for t in title:
  table[f'{t}']=table['Title'].map(lambda x: jaro(t, x))

In [180]:
table

,Title,aladdin,weathering,toy story,frozen ii,detective conan fist blue sapphire,lion king,avengers endgame,bohemian rhapsody,kingdom,one piece stampede,eiga doraemon nobita getsumen tansaki,masquerade hotel,joker,fly saitama,hit anyone one time,star wars episode ix rise skywalker,pokteumon mewtwo strikes back evolution,spider man far home,confidence man jp,pokteumon detective pikachu,fast furious presents hobbs shaw,godzilla king monsters,terminator dark fate,ossan love love dead,ralph breaks internet,green book,secret life pets,captain marvel,crayon shin chan honeymoon hurricane lost hiroshi,great war archimedes,whistleblower,fable,aquaman,maleficent mistress evil,kaguya sama love war,jtesani nin shinitai kodomo tachi,uta prince sama maji love kingdom movie,city hunter shinjuku private eyes,dragon ball super broly,...,kids slope,lady bird,phantom thread,detective conan darkest nightmare,honey,flea picking samurai,code geass lelouch rebellion ii transgression,liz blue bird,space battleship yamato chapter,money world,uso happyaku,downsizing,american renegades,eureka seven hi evolution anemone,dtc yukemuri junjou hen high low,pitch perfect,souvenir movie mariya takeuchi theater live,yakiniku dragon,get home wife always pretends dead,american assassin,waiting spring,wonderstruck,revenge girl,hotel transylvania summer vacation,fifty shades freed,uncle drew,infini force movie farewell gatchaman friend,big sick,stronger,sleepless,aprteis une histoire vraie,raw,last flag flying,flatliners,billionaire boys club,death stalin,emoji movie,living boy new york,wonder wheel,fathom premieres miracle mountain
0,aladdin,1.000000,0.559524,0.000000,0.335979,0.394180,0.576190,0.427579,0.423903,0.428571,0.298942,0.548622,0.538690,0.000000,0.547619,0.412698,0.476512,0.278999,0.507143,0.420635,0.402116,0.507440,0.507540,0.423810,0.410534,0.647186,0.414286,0.465608,0.619048,0.467143,0.458874,0.313187,0.436508,0.619048,0.577143,0.460317,0.394491,0.472997,0.448773,0.510559,...,0.317460,0.588624,0.476190,0.511204,0.000000,0.507143,0.468845,0.473016,0.483487,0.403175,0.408730,0.465079,0.514550,0.395382,0.506494,0.406593,0.387446,0.529365,0.391331,0.518908,0.542857,0.317460,0.317460,0.487115,0.420635,0.495238,0.386356,0.417989,0.422619,0.417989,0.513228,0.492063,0.427579,0.573810,0.503968,0.309524,0.408730,0.521645,0.317460,0.447090
1,weathering,0.559524,1.000000,0.462963,0.524074,0.546296,0.494444,0.530754,0.522176,0.000000,0.562169,0.565497,0.513889,0.522222,0.373016,0.615079,0.516266,0.512108,0.549206,0.500000,0.471340,0.528935,0.472222,0.563889,0.424242,0.551768,0.572222,0.562169,0.531746,0.483889,0.571549,0.467094,0.333333,0.484127,0.592222,0.507937,0.508987,0.495596,0.511364,0.533989,...,0.444444,0.416667,0.551587,0.560458,0.355556,0.549206,0.548700,0.450000,0.516577,0.450000,0.277778,0.494444,0.537037,0.526094,0.455628,0.533761,0.503367,0.522222,0.529915,0.562092,0.672222,0.472222,0.626984,0.560458,0.578704,0.494444,0.499015,0.398148,0.547222,0.462963,0.634568,0.444444,0.578373,0.588889,0.543651,0.662698,0.500000,0.424242,0.555556,0.555556
2,toy story,0.000000,0.462963,1.000000,0.481481,0.498148,0.307407,0.395833,0.503268,0.417989,0.555556,0.495712,0.398148,0.540741,0.576720,0.531217,0.430230,0.557184,0.383333,0.277778,0.446914,0.506944,0.486111,0.468519,0.458754,0.378788,0.474074,0.388889,0.515873,0.544021,0.378788,0.500712,0.425926,0.000000,0.585185,0.544974,0.437364,0.492502,0.560606,0.372786,...,0.416667,0.555556,0.637566,0.500871,0.540741,0.440741,0.553980,0.451852,0.505615,0.562963,0.509259,0.474074,0.277778,0.355219,0.435690,0.521368,0.502766,0.511111,0.440171,0.392157,0.570370,0.509259,0.462963,0.518363,0.611111,0.474074,0.343184,0.481481,0.569444,0.407407,0.462963,0.000000,0.481481,0.474074,0.531217,0.493386,0.416667,0.458754,0.416667,0.498148
3,frozen ii,0.335979,0.524074,0.481481,1.000000,0.514550,0.614815,0.481481,0.549891,0.335979,0.544444,0.511139,0.395833,0.533333,0.515873,0.464550,0.554054,0.495726,0.535185,0.555556,0.580247,0.5226

In [181]:
for c in table.columns:
  if c!='Title':
    table[c]=table[c].astype(float)

In [182]:
count=0
movie_series=[]
for c in table.columns:
  
  if c!='Title':
    #print(table[table[c]==1]['Title'].values[0])
    result=table[(table[c]>0.77)&(table[c]!=1)]['Title'].values
    if len(result) >0:
      print(f'-{c}-----')
      print(table[(table[c]>0.77)&(table[c]!=1)]['Title'].values)
    
    movie_series.extend(result)

-detective conan  fist  blue sapphire-----
['detective conan  darkest nightmare']
- lion king-----
['lion']
-star wars episode ix  rise  skywalker-----
['star wars episode viii  last jedi']
-spider man far  home-----
['spider man homecoming']
-godzilla king   monsters-----
['godzilla planet   monsters']
-crayon shin chan honeymoon hurricane  lost hiroshi-----
['crayon shin chan invasion alien shiriri']
-city hunter shinjuku private eyes-----
['city hunter']
-diner-----
['inferno']
-code geass lelouch    surrection-----
['code geass lelouch   rebellion ii transgression']
-pretty cure miracle universe-----
['pretty cure' 'pretty cure super stars']
-jumanji  next level-----
['jumanji welcome   jungle']
-fate stay night heaven  feel ii lost butterfly-----
['fate stay night heaven  feel  presage flower']
-john wick chapter parabellum-----
['john wick chapter']
-  wonderful family  wife  life-----
['  wonderful family']
- dog  journey-----
[' dog  purpose']
-fantastic beasts  crimes  grindel

In [183]:
movie_series=sorted(list(set(movie_series)))
movie_series

['   ',
 '   return',
 '  brave',
 '  breathe',
 '  corner   world',
 '  love',
 '  money   world',
 '  paris',
 '  rain',
 '  vanish',
 '  wonderful family',
 '  wonderful family  wife  life',
 '  x',
 ' 100th love  ',
 ' comes',
 ' commuter',
 ' dark tower',
 ' departures',
 ' dog  journey',
 ' dog  purpose',
 ' drive',
 ' equalizer',
 ' founder',
 ' glass castle',
 ' great wall',
 ' greatest showman',
 ' hero academia heroes rising',
 ' hero academia two heroes',
 ' lion king',
 ' love',
 ' predator',
 ' teacher',
 ' teacher  love',
 ' zero kara hajimeru isekai seikatsu  zero kara hajimeru isekai seikatsu memory snow',
 ' zero kara hajimeru isekai seikatsu hyouketsu  kizuna',
 'american assassin',
 'american made',
 'american renegades',
 'baby driver',
 'city hunter',
 'city hunter shinjuku private eyes',
 'code geass lelouch    surrection',
 'code geass lelouch   rebellion ii transgression',
 'crayon shin chan honeymoon hurricane  lost hiroshi',
 'crayon shin chan invasion alien s

In [184]:
word_counter_en = {}
for title in movie_series:
  for word in title.split(' '):
    # dict.get(key, 0)はdictにkeyがあればdict[key]を、なければ0を返すメソッドです
    word_counter_en[word] = word_counter_en.get(word, 0) + 1

In [185]:
word_counter_en

{'': 125,
 '100th': 1,
 'academia': 2,
 'aid': 3,
 'alien': 1,
 'american': 3,
 'anemone': 1,
 'assassin': 1,
 'baby': 1,
 'battleship': 3,
 'beasts': 2,
 'blue': 1,
 'bond': 1,
 'brave': 1,
 'breathe': 1,
 'build': 2,
 'butterfly': 1,
 'castle': 1,
 'ch': 1,
 'chan': 2,
 'chapter': 4,
 'choose': 1,
 'city': 2,
 'clash': 1,
 'code': 2,
 'comes': 1,
 'comet': 1,
 'commuter': 1,
 'conan': 4,
 'corner': 1,
 'crayon': 2,
 'crimes': 1,
 'crimson': 1,
 'cure': 3,
 'dark': 1,
 'darkest': 2,
 'das': 2,
 'departure': 1,
 'departures': 1,
 'detective': 4,
 'detonation': 1,
 'diner': 1,
 'dog': 2,
 'dr': 1,
 'drive': 1,
 'driver': 1,
 'eater': 1,
 'end': 1,
 'ending': 1,
 'enforcer': 1,
 'episode': 2,
 'equalizer': 1,
 'equals': 1,
 'eureka': 2,
 'evolution': 2,
 'ex': 3,
 'eyes': 1,
 'family': 2,
 'fantastic': 2,
 'far': 1,
 'fate': 2,
 'feel': 2,
 'final': 2,
 'finale': 2,
 'find': 1,
 'first': 1,
 'fist': 1,
 'flower': 1,
 'forever': 2,
 'founder': 1,
 'free': 2,
 'friends': 1,
 'geass': 2,
 '

In [186]:
def prepro_title(text):
  text = text.replace(':','')\
             .replace('：','')\
             .replace('劇場版','')\
             .replace('映画','')\
             .replace('\u3000',' ')\
             .replace('　',' ')\
             .replace('?',' ')\
             .replace('？',' ')\
             .replace('/','')
  return text

In [187]:
df['Title(ja)'] = df['Title(ja)'].dropna().apply(prepro_title)
table_ja = pd.DataFrame(df['Title(ja)'].dropna())

title = df['Title(ja)'].dropna().values
for t in title:
  table_ja[f'{t}']=table_ja['Title(ja)'].map(lambda x: jaro(t, x))
  table_ja[f'{t}']=table_ja[f'{t}'].astype(float)

In [188]:
movie_series_ja=[]

for c in table_ja.columns:
  
  if c!='Title(ja)':
    #print(table[table[c]==1]['Title'].values[0])
    result=table_ja[(table_ja[c]>0.70)&(table_ja[c]!=1)]['Title(ja)'].values
    if len(result) >0:
      print(f'-{c}-----')
      print(table_ja[(table_ja[c]>0.70)&(table_ja[c]!=1)]['Title(ja)'].values)
    
    movie_series_ja.extend(result)

-名探偵コナン 紺青の拳-----
['名探偵コナン から紅の恋歌' '名探偵コナン ゼロの執行人' '名探偵コナン 純黒の悪夢']
-アベンジャーズ／エンドゲーム-----
['アベンジャーズ／インフィニティ・ウォー']
-ドラえもん のび太の月面探査記-----
['ドラえもん のび太の南極カチコチ大冒険' 'ドラえもん のび太の宝島']
-スパイダーマンファー・フロム・ホーム-----
['スパイダーマンスパイダーバース' 'スパイダーマン ホームカミング']
-グリーンブック-----
['グリンチ' 'グリンチ']
-マレフィセント2-----
['マグニフィセント・セブン']
-コードギアス 復活のルルーシュ-----
['アス' 'コードギアス 反逆のルルーシュII 叛道（はんどう）']
-アリー スター誕生-----
['アス']
-スパイダーマンスパイダーバース-----
['スパイダーマンファー・フロム・ホーム']
- Fatestay night Heaven's Feel II. lost butterfly-----
[" Fatestay night Heaven's Feel I. presage flower"]
-ジョン・ウィックパラベラム-----
['ジョン・ウィックチャプター2']
-僕のワンダフル・ジャーニー-----
['僕のワンダフル・ライフ']
-IT／イット THE END “それ”が見えたら、終わり。-----
['IT／イット “それ”が見えたら、終わり。']
-ファンタスティック・ビーストと黒い魔法使いの誕生-----
['ファンタスティック・ビーストと魔法使いの旅']
-僕のヒーローアカデミア THE MOVIE ヒーローズライジング-----
['僕のヒーローアカデミア THE MOVIE 〜2人の英雄（ヒーロー）〜']
-仮面ライダー 令和 ザ・ファースト・ジェネレーション-----
['ライフ']
-それいけ！アンパンマン きらめけ！アイスの国のバニラ姫-----
['それいけ！アンパンマン かがやけ！クルンといのちの星']
-HiGH&LOW THE WORST-----
['HiGH&LOW THE MOVIE2  END OF SKY']
-グリンチ-----
['グリーンブック']
-アス---

In [189]:
movie_series_ja=sorted(list(set(movie_series_ja)))
movie_series_ja

[" Fatestay night Heaven's Feel I. presage flower",
 " Fatestay night Heaven's Feel II. lost butterfly",
 ' Free! Timeless Medley 約束',
 ' Free! Timeless Medley 絆',
 'ANEMONE／交響詩篇エウレカセブン ハイエボリューション',
 'GODZILLA 怪獣惑星',
 'GODZILLA 星を喰う者',
 'HiGH&LOW THE MOVIE2  END OF SKY',
 'HiGH&LOW THE WORST',
 'IT／イット THE END “それ”が見えたら、終わり。',
 'IT／イット “それ”が見えたら、終わり。',
 'Reゼロから始める異世界生活 Memory Snow',
 'Reゼロから始める異世界生活 氷結の絆',
 'T2 トレインスポッティング',
 'TAXi ダイヤモンド・ミッション',
 'それいけ！アンパンマン かがやけ！クルンといのちの星',
 'それいけ！アンパンマン きらめけ！アイスの国のバニラ姫',
 'それいけ！アンパンマン ブルブルの宝探し大冒険',
 'アサシン クリード',
 'アス',
 'アベンジャーズ／インフィニティ・ウォー',
 'アベンジャーズ／エンドゲーム',
 'アリー スター誕生',
 'アンクル・ドリュー',
 'インクレディブル・ファミリー',
 'キングスマンゴールデン・サークル',
 'キング・アーサー',
 'グリンチ',
 'グリーンブック',
 'グレイテスト・ショーマン',
 'コードギアス 反逆のルルーシュII 叛道（はんどう）',
 'コードギアス 復活のルルーシュ',
 'ジョン・ウィックチャプター2',
 'ジョン・ウィックパラベラム',
 'スパイダーマン ホームカミング',
 'スパイダーマンスパイダーバース',
 'スパイダーマンファー・フロム・ホーム',
 'ダークタワー',
 'ダーク・スクール',
 'トレイン・ミッション',
 'ドラえもん のび太の南極カチコチ大冒険',
 'ドラえもん のび太の宝島',
 'ドラえもん のび太の月面探査記',
 'ハン・ソロ／スター・ウォーズ・ストーリー',

In [190]:
word_counter = {}
for title in movie_series_ja:
  for word in title.split(' '):
    # dict.get(key, 0)はdictにkeyがあればdict[key]を、なければ0を返すメソッドです
    word_counter[word] = word_counter.get(word, 0) + 1

In [191]:
word_counter

{'': 5,
 'ANEMONE／交響詩篇エウレカセブン': 1,
 'Detonation': 1,
 'Dr.パックマン対エグゼイド＆ゴースト': 1,
 'END': 2,
 'FINAL': 1,
 'Fatestay': 2,
 'Feel': 2,
 'Free!': 2,
 'GODZILLA': 2,
 "Heaven's": 2,
 'HiGH&LOW': 2,
 'I.': 1,
 'II.': 1,
 'IT／イット': 2,
 'MOVIE': 4,
 'MOVIE2': 1,
 'Medley': 2,
 'Memory': 1,
 'OF': 1,
 'ORIGIN': 2,
 'Reflection': 1,
 'Reゼロから始める異世界生活': 2,
 'SKY': 1,
 'Snow': 1,
 'T2': 1,
 'TAXi': 1,
 'THE': 9,
 'Timeless': 2,
 'WORST': 1,
 'butterfly': 1,
 'flower': 1,
 'lost': 1,
 'night': 2,
 'presage': 1,
 'with': 1,
 '“それ”が見えたら、終わり。': 2,
 '〜2人の英雄（ヒーロー）〜': 1,
 'かがやけ！クルンといのちの星': 1,
 'から紅の恋歌': 1,
 'きらめけ！アイスの国のバニラ姫': 1,
 'それいけ！アンパンマン': 3,
 'のび太の南極カチコチ大冒険': 1,
 'のび太の宝島': 1,
 'のび太の月面探査記': 1,
 'みんなの物語': 1,
 'アサシン': 1,
 'アス': 1,
 'アベンジャーズ／インフィニティ・ウォー': 1,
 'アベンジャーズ／エンドゲーム': 1,
 'アリー': 1,
 'アンクル・ドリュー': 1,
 'インクレディブル・ファミリー': 1,
 'キミにきめた!': 1,
 'キングスマンゴールデン・サークル': 1,
 'キング・アーサー': 1,
 'クリード': 1,
 'グリンチ': 1,
 'グリーンブック': 1,
 'グレイテスト・ショーマン': 1,
 'コードギアス': 2,
 'ザ・ファースト・ジェネレーション': 1,
 'ジョン・ウィックチャプター2': 1,
 'ジ

In [193]:
#word_counter = sorted(word_counter.items())
#sortedで返ってくるのはリスト型なため辞書型に直す
word_counter_={}
for k, v in sorted(word_counter.items(), key=lambda x: -x[1]):
  word_counter_.update({k:v})
word_counter = pd.Series(word_counter_)

In [197]:
word_counter[word_counter>1]

THE                  9
                     5
MOVIE                4
名探偵コナン               4
それいけ！アンパンマン          3
ドラえもん                3
宇宙戦艦ヤマト2202          3
愛の戦士たち               3
Fatestay             2
night                2
Heaven's             2
Feel                 2
Free!                2
Timeless             2
Medley               2
GODZILLA             2
HiGH&LOW             2
END                  2
IT／イット               2
“それ”が見えたら、終わり。       2
Reゼロから始める異世界生活       2
コードギアス               2
ポケットモンスター            2
仮面ライダー平成ジェネレーションズ    2
僕のヒーローアカデミア          2
機動戦士ガンダム             2
ORIGIN               2
魔法少女リリカルなのは          2
dtype: int64